In [1]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
from pathlib import Path
import os.path
import time
from collections import OrderedDict
import csv
import shutil
import polars as pl
import fastexcel

In [2]:
def convert_to_datetime(struct_time):
    return datetime(*struct_time[:6])

def unify_dtypes(dfs):
    all_columns = set()
    for df in dfs:
        all_columns.update(df.columns)

    col_dtypes = {}
    for col in all_columns:
        dtypes = set()
        for df in dfs:
            if col in df.columns:
                dtypes.add(df[col].dtype)
        col_dtypes[col] = dtypes

    new_dfs = []
    for df in dfs:
        df_new = df
        for col, dtypes in col_dtypes.items():
            if col in df_new.columns:
                if len(dtypes) > 1:
                    df_new = df_new.with_columns(pl.col(col).cast(pl.Utf8))
                else:
                    pass
        new_dfs.append(df_new)
    return new_dfs

def input_data(data_dir):
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.*'):
        file_suffixes = filename.suffixes
        if file_suffixes[-1].lower() in ['.xlsx', '.csv']:
            try:
                export_time = os.path.getmtime(filename)
                export_time_datetime = convert_to_datetime(time.localtime(export_time))
                file_name = filename.stem

                if file_suffixes[-1].lower() == '.xlsx':
                    df = pl.read_excel(filename)
                else:
                    df = pl.read_csv(filename)

                df = df.with_columns([
                    pl.lit(file_name).alias('sheet_name'),
                    pl.lit(export_time_datetime).alias('Export time')
                ])

                list_files.append(df)

            except Exception as e:
                print(f"Error reading file {filename}: {e}")

    if list_files:
        list_files = unify_dtypes(list_files)
        df_list = pl.concat(list_files)
        return df_list
    else:
        return pl.DataFrame()

    
def create_datetime(row, time_column, shift_column=None):
    if row['Scheduled'] == 0:
        return pd.NaT
    if pd.isnull(row['Date']) or pd.isnull(row[time_column]):
        return pd.NaT
    time_value = row[time_column]
    if shift_column and pd.isnull(row[shift_column]):
        return pd.NaT
    if shift_column and time_value < row[shift_column]:
        return pd.Timestamp.combine(row['Date'], time_value) + pd.DateOffset(days=1)
    return pd.Timestamp.combine(row['Date'], time_value)

def convert_to_time(df, columns):
    for column in columns:
        df[column] = pd.to_datetime(df[column], format='%I:%M %p', errors='coerce').dt.time
    return df

def combine_date_and_time(row, time_column):
    return pd.to_datetime(f"{row['Date'].date()} {row[time_column]}")

In [3]:
first_glob_1 = "C:/Users/huuchinh.nguyen"
first_glob_2 = "C:/Users/ADMIN"

if os.path.exists(first_glob_1):
    first_glob = first_glob_1
elif os.path.exists(first_glob_2):
    first_glob = first_glob_2
else:
    raise FileNotFoundError(f"Neither {first_glob_1} nor {first_glob_2} exists.")

folder_paths = {
    "input_iex":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_AGENT_IEX_FOR_REPORT',
    "input_hc_master":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Master Database - 2025.xlsx',
    "output_iex":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/OUTPUT_AGENT_IEX_FOR_REPORT',
    "output_iex_intervals":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/OUTPUT_AGENT_IEX_INTERVALS',
    "hc_extend_by_month":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Extend by Month'
}

In [4]:
IEX_Input = input_data(folder_paths["input_iex"])

IEX_Input = IEX_Input.to_pandas()

HC_MASTER_DATABASE = input_data(folder_paths["hc_extend_by_month"])
HC_MASTER_DATABASE = HC_MASTER_DATABASE.to_pandas()
HC_MASTER_DATABASE = HC_MASTER_DATABASE.rename(columns={'IEX ID': 'IEX_ID'})

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 8, falling back to string
Could not determine dtype for column 63, falling back to string
Could not determine dtype for column 64, falling back to string
Could not determine dtype for column 65, falling back to string
Could not determine dtype for column 63, falling bac

In [5]:
columns_to_remove = ["__UNNAMED__4"]

IEX = IEX_Input[[col for col in IEX_Input.columns if col not in columns_to_remove]]
IEX = IEX.rename(columns={
    "Agent Schedules": "Agent",
    "__UNNAMED__1": "Date",
    "__UNNAMED__2": "Start_Shift",
    "__UNNAMED__3": "End_Shift",
    "__UNNAMED__5": "Scheduled Activity",
    "__UNNAMED__6": "Start_Action",
    "__UNNAMED__9": "End_Action"
})

IEX['Generate Date'] = np.where(
    IEX['Agent'].str.contains("Generation Date: ", na=False),
    IEX['Agent'].str.extract(r'Generation Date: (.+)')[0],
    np.nan
)
IEX['Generate Date'] = IEX['Generate Date'].fillna(method='bfill')
IEX_edit = IEX.copy()
IEX_edit['Agent'] = IEX_edit['Agent'].fillna(method='ffill')
offTable_save = IEX_edit
IEX_edit = IEX_edit[(IEX_edit['Start_Shift'] != "Off") & (IEX_edit['Date'] != "Date")]
IEX_edit = IEX_edit[~(IEX_edit['Date'].isna() & IEX_edit['Scheduled Activity'].isna())]
IEX_edit

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18108\3870183097.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX['Generate Date'] = IEX['Generate Date'].fillna(method='bfill')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18108\3870183097.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX_edit['Agent'] = IEX_edit['Agent'].fillna(method='ffill')


,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date
4,"Agent: 3052306 BUI, BADUONG",5/12/25,7:00 AM,4:00 PM,Open Time,7:00 AM,9:45 AM,VNM_12_May_18_May_2025,2025-05-20 11:55:27,5/20/25 4:54 AM
5,"Agent: 3052306 BUI, BADUONG",None,None,None,Break,9:45 AM,10:00 AM,VNM_12_May_18_May_2025,2025-05-20 11:55:27,5/20/25 4:54 AM
6,"Agent: 3052306 BUI, BADUONG",None,None,None,Open Time,10:00 AM,12:40 PM,VNM_12_May_18_May_2025,2025-05-20 11:55:27,5/20/25 4:54 AM
7,"Agent: 3052306 BUI, BADUONG",None,None,None,Lunch,12:40 PM,1:40 PM,VNM_12_May_18_May_2025,2025-05-20 11:55:27,5/20/25 4:54 AM
8,"Agent: 3052306 BUI, BADUONG",None,None,None,Open Time,1:40 PM,2:40 PM,VNM_12_May_18_May_2025,2025-05-20 11:55:27,5/20/25 4:54 AM
...,...,...,...,...,...,...,...,...,...,...
42459,"Agent: 3089155 Vuu, TienNhan",None,None,None,Open Time,8:45 AM,11:25 AM,VNM_9_June_15_Jun_2025,2025-06-14 11:30:46,6/14/25 4:29 AM
42460,"Agent: 3089155 Vuu, TienNhan",None,None,None,Lunch,11:25 AM,12:25 PM,VNM_9_June_15_Jun_2025,2025-06-14 11:30:46,6/14/25 4:29 AM
42461,"Agent: 3089155 Vuu, TienNhan",None,None,None,Open Time,12:25 PM,1:25 PM,VNM_9_June_15_Jun_2025,2025-06-14 11:30:46,6/14/25 4:29 AM
42462,"Agent: 3089155 Vuu, TienNhan",None,None,None,Break,1:25 PM,1:40 PM,VNM_9_June_15_Jun_2025,2025-06-14 11:30:46,6/14/25 4:29 AM


In [6]:
offTable = offTable_save[offTable_save['Start_Shift'] == "Off"].reset_index(drop=True)

condition = (IEX_edit['Date'] != "Date") & (IEX_edit['Start_Shift'] != "Off")
IEX_edit = IEX_edit[condition].reset_index(drop=True)
IEX_edit[['Date', 'Start_Shift', 'End_Shift']] = IEX_edit[['Date', 'Start_Shift', 'End_Shift']].fillna(method='ffill')

condition2 = (IEX_edit['Agent'].str.contains("Agent: ", na=False)) | (IEX_edit['Agent'].isna())
IEX_edit = IEX_edit[condition2].reset_index(drop=True)

offTable['Scheduled Activity'] = offTable['Scheduled Activity'].fillna(offTable['Start_Shift'])
offTable['Start_Shift'] = offTable['Start_Shift'].replace('Off', np.nan)

IEX_edit = IEX_edit[IEX_edit['Scheduled Activity'].notna()].reset_index(drop=True)
combined_df = pd.concat([offTable, IEX_edit], axis=0, ignore_index=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18108\1879448329.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX_edit[['Date', 'Start_Shift', 'End_Shift']] = IEX_edit[['Date', 'Start_Shift', 'End_Shift']].fillna(method='ffill')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18108\1879448329.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  offTable['Start_Shift'] = offTable['Start_Shift'].replace('Off', np.nan)


In [7]:
combined_df['Export time'] = pd.to_datetime(combined_df['Export time'], errors='coerce')
combined_df['Generate Date'] = pd.to_datetime(combined_df['Generate Date'], errors='coerce')
combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Month'] = combined_df['Date'].dt.strftime('%b-%y')
combined_df['IEX_ID'] = combined_df['Agent'].str.extract(r'(\d+)', expand=False).astype(int)

max_generate_date_df = combined_df.groupby(['IEX_ID','Date'])['Generate Date'].max().reset_index()
max_generate_date_df = max_generate_date_df.rename(columns={'Generate Date': 'Max Generate Date'})

max_generate_date_with_details = pd.merge(combined_df, max_generate_date_df, on=['Date','IEX_ID'], how='left')
filtered_max_generate_date = max_generate_date_with_details[max_generate_date_with_details['Generate Date'] == max_generate_date_with_details['Max Generate Date']]

filtered_max_generate_date['Scheduled'] = filtered_max_generate_date.groupby(['Date', 'IEX_ID'])['Scheduled Activity'].transform(lambda x: 1 if any(x.isin(['Open Time', 'Extra Hours', 'No Call/No Show', 'PTO', 'Training Offline', 'Sick Leave', 'Paid Leave', 'Termination', 'Off Phone Misc','Billable Training'])) else 0)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18108\48420382.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_df['Generate Date'] = pd.to_datetime(combined_df['Generate Date'], errors='coerce')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18108\48420382.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')


In [23]:
time_columns = ['Start_Shift', 'End_Shift', 'Start_Action', 'End_Action']
filtered_max_generate_date = convert_to_time(filtered_max_generate_date, time_columns)

# filtered_max_generate_date = filtered_max_generate_date.drop(filtered_max_generate_date[(filtered_max_generate_date['Scheduled Activity'].isin(['Lunch', 'Break','Training'])) & (filtered_max_generate_date['Scheduled'] == 0)].index)

filtered_max_generate_date['Datetime_Start_Action'] = filtered_max_generate_date.apply(lambda row: create_datetime(row, 'Start_Action', 'Start_Shift'), axis=1)
filtered_max_generate_date['Datetime_End_Action'] = filtered_max_generate_date.apply(lambda row: create_datetime(row, 'End_Action', 'Start_Shift'), axis=1)

filtered_fluctuate_shifts = filtered_max_generate_date[
    filtered_max_generate_date['Scheduled Activity'].isin(["Open Time", "Extra Hours"]) | 
    filtered_max_generate_date['Scheduled Activity'].str.contains("Training", na=False)
]
min_max_fluctuate_shifts = filtered_fluctuate_shifts.groupby(['Date', 'IEX_ID']).agg(
    Datetime_Fluctuate_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_Fluctuate_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_fluctuate_shifts['Fluctuate Shift'] = (min_max_fluctuate_shifts['Datetime_Fluctuate_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_fluctuate_shifts['Datetime_Fluctuate_End_Shift'].dt.strftime('%H%M'))
new_table_with_fluctuate_shifts_col = pd.merge(filtered_max_generate_date, min_max_fluctuate_shifts, on=['Date', 'IEX_ID'], how='left')

min_max_first_shifts = filtered_max_generate_date.groupby(['Date', 'IEX_ID']).agg(
    Datetime_First_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_First_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_first_shifts['First Shift'] = (min_max_first_shifts['Datetime_First_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_first_shifts['Datetime_First_End_Shift'].dt.strftime('%H%M'))
new_table_with_first_shifts_col = pd.merge(new_table_with_fluctuate_shifts_col, min_max_first_shifts, on=['Date', 'IEX_ID'], how='left')

filtered_original_shifts = filtered_max_generate_date[
    filtered_max_generate_date['Scheduled Activity'].isin(["Open Time"]) | 
    filtered_max_generate_date['Scheduled Activity'].str.contains("Training", na=False)
]
min_max_original_shifts = filtered_original_shifts.groupby(['Date', 'IEX_ID']).agg(
    Datetime_Original_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_Original_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_original_shifts['Original Shift'] = (min_max_original_shifts['Datetime_Original_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_original_shifts['Datetime_Original_End_Shift'].dt.strftime('%H%M'))
new_table_with_original_shifts_col = pd.merge(new_table_with_first_shifts_col, min_max_original_shifts, on=['Date', 'IEX_ID'], how='left')

new_table_with_original_shifts_col = new_table_with_original_shifts_col[
    ~(
        (new_table_with_original_shifts_col['Scheduled Activity'].isin(["Lunch", "Break"])) & 
        new_table_with_original_shifts_col['Fluctuate Shift'].isna()
    )
]

In [24]:
sorted_generate_date = new_table_with_original_shifts_col.sort_values(by='Export time', ascending=True)

sorted_generate_date['Night_Shift'] = sorted_generate_date['Datetime_First_Start_Shift'].apply(lambda x: 1 if pd.to_datetime(x).hour >= 15 else 0)
sorted_generate_date['Week_Monday'] = sorted_generate_date['Date'] - pd.to_timedelta(sorted_generate_date['Date'].dt.weekday, unit='d')
sorted_generate_date['Week Begin'] = sorted_generate_date['Week_Monday'].dt.strftime('WB%d%m')
sorted_generate_date['Agent Name'] = sorted_generate_date['Agent'].str.extract(r'\d+ (.+)', expand=False).str.upper()
sorted_generate_date['Duration'] = (sorted_generate_date['Datetime_End_Action'] - sorted_generate_date['Datetime_Start_Action']).dt.total_seconds()

def calculate_total_time(df, activity_list, time_column_name):
    time_totals = df[df['Scheduled Activity'].isin(activity_list)]
    time_totals = time_totals.groupby(['Date', 'IEX_ID'])['Duration'].sum().reset_index()
    time_totals.rename(columns={'Duration': time_column_name}, inplace=True)
    time_totals[time_column_name] = time_totals[time_column_name] / 3600
    return time_totals

sorted_generate_date = sorted_generate_date.drop_duplicates()

activities = {
    'Open Time': 'Open Time',
    'Break': 'Break Time',
    'Lunch': 'Lunch Time',
    'Extra Hours': 'Extra Time',
    'No Call/No Show': 'NCNS',
    'Training Offline': 'Training',
    'Training': 'Training',
    'Training Nesting': 'Training',
    'PTO':'AL'
}
training_activities = ['Training Offline', 'Training', 'Training Nesting']
time_totals_training = calculate_total_time(sorted_generate_date, training_activities, 'Training')
sorted_generate_date = sorted_generate_date.merge(time_totals_training, on=['Date', 'IEX_ID'], how='left')

for activity, column_name in activities.items():
    if activity not in training_activities:
        time_totals = calculate_total_time(sorted_generate_date, [activity], column_name)
        sorted_generate_date = sorted_generate_date.merge(time_totals, on=['Date', 'IEX_ID'], how='left')

sorted_generate_date['Target'] = sorted_generate_date.apply(
    lambda row: row['Open Time'] if pd.isnull(row['Extra Time']) else
                 row['Extra Time'] if pd.isnull(row['Open Time']) else
                 row['Open Time'] + row['Extra Time'],axis=1)

sorted_generate_date['Time_Of_Day'] = ((sorted_generate_date['Datetime_First_End_Shift'] - sorted_generate_date['Datetime_First_Start_Shift']).dt.total_seconds())/3600
sorted_generate_date = sorted_generate_date.sort_values(by=['Date', 'IEX_ID', 'Datetime_Start_Action'], na_position='last')
sorted_generate_date['First_Scheduled_Activity'] = sorted_generate_date.groupby(['Date', 'IEX_ID'])['Scheduled Activity'].transform('first')

conditions = [
    ((sorted_generate_date['Open Time'] > 0) & (sorted_generate_date['NCNS'] > 3)) | (sorted_generate_date['NCNS'] <= 5),
    ((sorted_generate_date['Open Time'] > 0) | (sorted_generate_date['Open Time'].isna())) & (sorted_generate_date['NCNS'] > 5),
    (sorted_generate_date['Open Time'] > 0) & (sorted_generate_date['Extra Time'] > 0),
    (sorted_generate_date['Open Time'] == 0) & (sorted_generate_date['Extra Time'] > 0),
    (sorted_generate_date['Open Time'] > 0),
    (sorted_generate_date['First_Scheduled_Activity'] == "Extra Hours") & (sorted_generate_date['Open Time'].isna()),
    sorted_generate_date['First_Scheduled_Activity'].isin(["Holiday", "Bereavement", "Off", "Off Phone Misc", "Termination", "Unscheduled"]),
    (sorted_generate_date['First_Scheduled_Activity'] == "PTO"),
    ((sorted_generate_date['First_Scheduled_Activity'] == "Sickness") | (sorted_generate_date['First_Scheduled_Activity'] == "Sick Leave")),
    sorted_generate_date['First_Scheduled_Activity'].isin(["Training Offline"]),
    sorted_generate_date['First_Scheduled_Activity'].isin(["Paid Leave"]),
    sorted_generate_date['First_Scheduled_Activity'].isin(["Leave"]),
    sorted_generate_date['First_Scheduled_Activity'].isin(["Termination"]),
]

choices = [
    "HDL",
    "NCNS",
    "PR - OT",
    "PO",
    "PR",
    "PO",
    sorted_generate_date['First_Scheduled_Activity'],
    "AL",
    "SL",
    "Training Offline",
    "CO",
    "LWP",
    "Termination"
]

sorted_generate_date['Shift Tracking'] = np.select(conditions, choices, default=np.nan)

sorted_generate_date['Fluctuate Shift'] = np.where(sorted_generate_date['Time_Of_Day'].isnull(), sorted_generate_date['Scheduled Activity'], sorted_generate_date['Fluctuate Shift'])
sorted_generate_date['First Shift'] = np.where(sorted_generate_date['Time_Of_Day'].isnull(), sorted_generate_date['Scheduled Activity'], sorted_generate_date['First Shift'])

ot_range_tracker = sorted_generate_date

ot_range_tracker['OT Range'] = ot_range_tracker.apply(
    lambda row: row['Fluctuate Shift'] if row['Shift Tracking'] == 'PO' else (
        f"{row['Datetime_Start_Action'].strftime('%H%M')}-{row['Datetime_End_Action'].strftime('%H%M')}" 
        if row['Scheduled Activity'] == 'Extra Hours' else None
    ), axis=1)

def combine_ot_range(group):
    non_null_ot_range = [val for val in group if val is not None and val != ""]
    distinct_ot_range = list(set(non_null_ot_range))
    return ",".join(distinct_ot_range) if distinct_ot_range else None

ot_range_tracker['Combined OT Range'] = (ot_range_tracker.groupby(['Date', 'IEX_ID'])['OT Range'].transform(combine_ot_range))
ot_range_tracker = ot_range_tracker[['Date','IEX_ID','Combined OT Range']]

In [25]:
def calculate_ot_pre_post_shift(df):
    extra_hours_df = df[df['Scheduled Activity'] == 'Extra Hours']
    
    ot_df = df[['Date', 'IEX_ID']].drop_duplicates()
    
    def calculate_duration_for_group(group_df, date, iex_id, shift_start, shift_end):
        pre_shift_duration = extra_hours_df[
            (extra_hours_df['Date'] == date) & 
            (extra_hours_df['IEX_ID'] == iex_id) & 
            (extra_hours_df['Datetime_Start_Action'] < shift_start)
        ]['Duration'].sum()
        
        post_shift_duration = extra_hours_df[
            (extra_hours_df['Date'] == date) & 
            (extra_hours_df['IEX_ID'] == iex_id) & 
            (extra_hours_df['Datetime_End_Action'] > shift_end)
        ]['Duration'].sum()
        
        return pre_shift_duration, post_shift_duration
    
    ot_df[['OT PreShift Seconds', 'OT PostShift Seconds']] = ot_df.apply(
        lambda row: pd.Series(
            calculate_duration_for_group(
                extra_hours_df, 
                row['Date'], 
                row['IEX_ID'], 
                df[(df['Date'] == row['Date']) & (df['IEX_ID'] == row['IEX_ID'])]['Datetime_Original_Start_Shift'].min(),
                df[(df['Date'] == row['Date']) & (df['IEX_ID'] == row['IEX_ID'])]['Datetime_Original_End_Shift'].max()
            )
        ), 
        axis=1
    )
    ot_df['OT PreShift'] = ot_df['OT PreShift Seconds'] / 3600
    ot_df['OT PostShift'] = ot_df['OT PostShift Seconds'] / 3600

    ot_df = ot_df.drop(columns=['OT PreShift Seconds', 'OT PostShift Seconds'])
    df = df.merge(ot_df, on=['Date', 'IEX_ID'], how='left')
    
    return df

OT_Pre_Post_Table = calculate_ot_pre_post_shift(sorted_generate_date)
OT_Pre_Post_Table = OT_Pre_Post_Table.groupby(['Date', 'IEX_ID'])[['OT PreShift', 'OT PostShift']].max().reset_index()
combine_OT_Pre_Post_Table = pd.merge(sorted_generate_date, OT_Pre_Post_Table, on=['Date','IEX_ID'], how='left')
combine_OT_Pre_Post_Table['Start Date'] = pd.to_datetime(combine_OT_Pre_Post_Table['Datetime_Start_Action']).dt.normalize()
combine_OT_Pre_Post_Table['End Date'] = pd.to_datetime(combine_OT_Pre_Post_Table['Datetime_End_Action']).dt.normalize()

combine_OT_Pre_Post_Table

,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date,...,Target,Time_Of_Day,First_Scheduled_Activity,Shift Tracking,OT Range,Combined OT Range,OT PreShift,OT PostShift,Start Date,End Date
0,"Agent: 3000182 HOANG, THIPHUONGHOA",2025-05-05,21:00:00,06:00:00,Open Time,21:00:00,22:15:00,VNM_5_May_11_May_2025,2025-05-14 06:37:10,2025-05-13 23:35:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-05-05,2025-05-05
1,"Agent: 3000182 HOANG, THIPHUONGHOA",2025-05-05,21:00:00,06:00:00,Break,22:15:00,22:30:00,VNM_5_May_11_May_2025,2025-05-14 06:37:10,2025-05-13 23:35:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-05-05,2025-05-05
2,"Agent: 3000182 HOANG, THIPHUONGHOA",2025-05-05,21:00:00,06:00:00,Open Time,22:30:00,01:30:00,VNM_5_May_11_May_2025,2025-05-14 06:37:10,2025-05-13 23:35:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-05-05,2025-05-06
3,"Agent: 3000182 HOANG, THIPHUONGHOA",2025-05-05,21:00:00,06:00:00,Lunch,01:30:00,02:30:00,VNM_5_May_11_May_2025,2025-05-14 06:37:10,2025-05-13 23:35:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-05-06,2025-05-06
4,"Agent: 3000182 HOANG, THIPHUONGHOA",2025-05-05,21:00:00,06:00:00,Open Time,02:30:00,03:45:00,VNM_5_May_11_May_2025,2025-05-14 06:37:10,2025-05-13 23:35:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-05-06,2025-05-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31852,"Agent: 3097238 Nguyen, LamNgocLong",2025-06-08,20:00:00,05:00:00,Lunch,00:00:00,01:00:00,VNM_2_June_8_Jun_2025,2025-06-08 12:01:59,2025-06-08 05:01:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-06-09,2025-06-09
31853,"Agent: 3097238 Nguyen, LamNgocLong",2025-06-08,20:00:00,05:00:00,Open Time,01:00:00,03:25:00,VNM_2_June_8_Jun_2025,2025-06-08 12:01:59,2025-06-08 05:01:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-06-09,2025-06-09
31854,"Agent: 3097238 Nguyen, LamNgocLong",2025-06-08,20:00:00,05:00:00,Break,03:25:00,03:40:00,VNM_2_June_8_Jun_2025,2025-06-08 12:01:59,2025-06-08 05:01:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-06-09,2025-06-09
31855,"Agent: 3097238 Nguyen, LamNgocLong",2025-06-08,20:00:00,05:00:00,Open Time,03:40:00,05:00:00,VNM_2_June_8_Jun_2025,2025-06-08 12:01:59,2025-06-08 05:01:00,...,7.5,9.0,Open Time,PR,None,NaN,0.0,0.0,2025-06-09,2025-06-09


In [26]:
iex_ids_with_max_export_time = combine_OT_Pre_Post_Table[['sheet_name', 'Generate Date','Year','Month','Week Begin','Week_Monday','Date','Start Date','End Date','Agent Name', 'IEX_ID','Datetime_Fluctuate_Start_Shift',
       'Datetime_Fluctuate_End_Shift', 'Fluctuate Shift',
       'Datetime_First_Start_Shift', 'Datetime_First_End_Shift',
       'First Shift','Scheduled Activity','Datetime_Start_Action','Datetime_End_Action','Time_Of_Day','Duration','Open Time','Extra Time','Target','Break Time','Lunch Time','Training','NCNS','AL','Night_Shift', 'Scheduled','Shift Tracking','OT PreShift', 'OT PostShift','Combined OT Range']]
iex_merged_with_HC = pd.merge(
    iex_ids_with_max_export_time,
    HC_MASTER_DATABASE[['Date','OracleID','People ID', 'IEX_ID','Employee Name','Email Id','Alias','LOB','LOB_2','LOB_3','Supervisor Name','Wave','Detail Status','Status']],
    on=['Date','IEX_ID'],
    how='left'
)

sorted_df = iex_merged_with_HC.sort_values(by=[ 'Date','IEX_ID','Datetime_Start_Action'],ascending=[ True, True,True]).reset_index(drop=True)

sorted_df = sorted_df.drop_duplicates()

def label_breaks(group):
    group.loc[group['Scheduled Activity'] == 'Break', 'Scheduled Activity'] = (
        group['Scheduled Activity'] + '_' + (group['Scheduled Activity'].eq('Break').cumsum()).astype(str))
    return group

sorted_df = sorted_df.groupby(['Date', 'IEX_ID']).apply(label_breaks)
sorted_df = sorted_df.reset_index(drop=True)

fill_zero = ['Time_Of_Day', 'Open Time', 'Extra Time', 'Target', 'Break Time', 'Lunch Time', 'Training', 'NCNS', 'AL', 'Night_Shift', 'OT PreShift', 'OT PostShift']
sorted_df[fill_zero] = sorted_df[fill_zero].fillna(0)

sorted_df['Roster Scheduled'] = sorted_df.apply(lambda x: 0.5 if (0 <= x['Time_Of_Day'] < 6) and (x['Scheduled'] == 1) else x['Scheduled'], axis=1)
sorted_df['Roster Presented'] = sorted_df.apply(lambda x: 0.5 if 0 < (x['Open Time'] + x['Extra Time'] + x['Training']) < 6 else (0 if (x['Open Time'] + x['Extra Time'] + x['Training']) == 0 else 1),axis=1)

sorted_df['Unplanned'] = None
sorted_df['Unplanned'] = sorted_df.apply(lambda x: 0.5 if 0 < x['NCNS'] < 6 else (1 if x['NCNS'] >= 6 else None), axis=1)

sorted_df['Planned'] = None
sorted_df['Planned'] = sorted_df.apply(lambda x: 0.5 if 0 < x['AL'] < 6 else (1 if x['AL'] >= 6 else None), axis=1)

fill_zero2 = ['Unplanned', 'Planned', 'Roster Presented', 'Roster Scheduled']
sorted_df[fill_zero2] = sorted_df[fill_zero2].fillna(0)

def determine_ot_type(row):
    if row["Shift Tracking"] == "PO": return "OT - PO"
    elif row["Shift Tracking"] == "PR - OT":
        if (row["OT PreShift"] > 0) and (row["OT PostShift"] > 0): return "OT - Pre/Post Shift"
        elif (row["OT PreShift"] > 0): return "OT - Pre Shift"
        elif (row["OT PostShift"] > 0): return "OT - Post Shift"
        else: return None
    else: return "No OT"

sorted_df["OT Type"] = sorted_df.apply(determine_ot_type, axis=1)

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_19024\2603243299.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = sorted_df.groupby(['Date', 'IEX_ID']).apply(label_breaks)


In [27]:
split_table = sorted_df[['Year','Month','Week_Monday','Date','OracleID','People ID', 'IEX_ID', 'Employee Name','Email Id','Alias','LOB','LOB_3', 'Supervisor Name','Wave','Detail Status','Status','Start Date', 'End Date', 'Scheduled Activity', 'Datetime_Start_Action', 'Datetime_End_Action']]

for col in ['Start', 'End']:
    split_table[f'{col}_Time_Hours'] = split_table[f'Datetime_{col}_Action'].dt.hour + \
                                       split_table[f'Datetime_{col}_Action'].dt.minute / 60 + \
                                        split_table[f'Datetime_{col}_Action'].dt.second / 3600
    split_table[f'{col}_Time_Hours'] = split_table[f'{col}_Time_Hours'].round(2)

split_table['End_Time_Hours'] = split_table.apply(
    lambda row: row['End_Time_Hours'] + 24 if row['End_Time_Hours'] < row['Start_Time_Hours'] else row['End_Time_Hours'],
    axis=1
)

for col in ['Start', 'End']:
    split_table[f'{col}_Time_Half_Rounding'] = split_table[f'{col}_Time_Hours'].apply(
        lambda x: np.round(x * 2) / 2 if np.round(x * 2) / 2 == x else np.floor(x * 2) / 2
    )

split_table['Number_Split'] = split_table.apply(
    lambda row: (row['End_Time_Half_Rounding'] - row['Start_Time_Half_Rounding']) * 2 
    if pd.notna(row['End_Time_Half_Rounding']) and pd.notna(row['Start_Time_Half_Rounding']) 
    else np.nan,
    axis=1
)

split_table = split_table.rename(columns={'Start_Time_Half_Rounding': 'Time_1'})
split_table['Time_2'] = split_table['Time_1'] + 0.5
split_table['Custom'] = split_table['Number_Split'].apply(lambda n: list(range(int(n) + 1)) if pd.notna(n) else [])
split_table = split_table.explode('Custom')

print(split_table['Custom'].unique())

split_table['Time_1_New'] = split_table['Time_1'] + (split_table['Custom'] * 0.5)
split_table['Time_2_New'] = split_table['Time_2'] + (split_table['Custom'] * 0.5)
split_table['Rounded_End_Time_Hours'] = np.ceil(split_table['End_Time_Hours'] * 2) / 2

filtered_df = split_table[split_table['Rounded_End_Time_Hours'] > split_table['Time_1_New']]

split_table['Start_Time_Number'] = np.where(
    split_table['Time_1_New'] > split_table['Start_Time_Hours'],
    split_table['Time_1_New'],
    split_table['Start_Time_Hours']
)

split_table['End_Time_Number'] = np.where(
    split_table['Time_2_New'] < split_table['End_Time_Hours'],
    split_table['Time_2_New'],
    split_table['End_Time_Hours']
)

def calculate_new_day_time(row, time_col, date_col):
    calculated_value = row[time_col]
    if row['Start Date'] == row['End Date']:
        return row[date_col]
    else:
        if time_col == 'Start_Time_Number' and calculated_value >= 24:
            return row['Start Date'] + pd.Timedelta(days=1)
        elif time_col == 'End_Time_Number' and calculated_value < 24:
            return row['End Date'] - pd.Timedelta(days=1)
        else:
            return row[date_col]

split_table['New_Day_Start_Time'] = split_table.apply(lambda row: calculate_new_day_time(row, 'Start_Time_Number', 'Start Date'), axis=1)
split_table['New_Day_End_Time'] = split_table.apply(lambda row: calculate_new_day_time(row, 'End_Time_Number', 'End Date'), axis=1)

for col in ['Start', 'End']:
    split_table[f'{col}_Time_Number'] = (split_table[f'{col}_Time_Number'] - 24 * (split_table[f'{col}_Time_Number'] >= 24)) % 24
              
def number_to_time(number):
    if pd.isna(number):
        return pd.NaT
    hours = int(np.floor(number))
    minutes = int(np.floor((number - hours) * 60))
    seconds = int(np.floor((number - hours - minutes / 60) * 3600))
    return pd.Timestamp(f"1970-01-01 {hours:02}:{minutes:02}:{seconds:02}")

for col in ['Start', 'End']:
    split_table[f'{col}_Time'] = split_table[f'{col}_Time_Number'].apply(number_to_time).dt.time
split_table['Time_1'] = split_table['Time_1'].apply(number_to_time).dt.time

def combine_date_time(row, time_col, date_col):
    if pd.isna(row[time_col]):
        return pd.NaT
    return datetime.combine(row[date_col].date(), row[time_col])

split_table['Datetime_Start_Time'] = split_table.apply(lambda row: combine_date_time(row, 'Start_Time', 'New_Day_Start_Time'), axis=1)
split_table['Datetime_End_Time'] = split_table.apply(lambda row: combine_date_time(row, 'End_Time', 'New_Day_End_Time'), axis=1)
split_table['Duration'] = (split_table['Datetime_End_Time'] - split_table['Datetime_Start_Time']).dt.total_seconds()
split_table['Work Category'] = np.where(split_table['Scheduled Activity'].isin(['Open Time', 'Extra Hours']),'Productive','Unproductive')

def adjust_time_to_interval(row, start_time_col):
    dt = row[start_time_col]
    hour = dt.hour
    minute = dt.minute
    if minute < 30:
        adjusted_time = dt.replace(minute=0, second=0, microsecond=0)
    else:
        adjusted_time = dt.replace(minute=30, second=0, microsecond=0)
        
    return adjusted_time

split_table['Intervals'] = split_table.apply(lambda row: adjust_time_to_interval(row, 'Datetime_Start_Time'), axis=1)

split_table = split_table[split_table['Datetime_Start_Time'] < split_table['Datetime_End_Time']]
INTERVALS_EXTEND = split_table[['Year','Month','Week_Monday','Date','OracleID','People ID', 'IEX_ID', 'Employee Name','Email Id', 'Alias','LOB','LOB_3', 'Supervisor Name','Wave','Detail Status','Status','Start Date', 'End Date', 'Scheduled Activity','Intervals', 'Datetime_Start_Time', 'Datetime_End_Time','Duration','Work Category']]

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_19024\2715543524.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_table[f'{col}_Time_Hours'] = split_table[f'Datetime_{col}_Action'].dt.hour + \
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_19024\2715543524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_table[f'{col}_Time_Hours'] = split_table[f'{col}_Time_Hours'].round(2)
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_19024\2715543524.py:4: SettingWithCopyWarning: 
A 

[0 1 2 3 4 5 6 nan 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [28]:
SCHEDULE = sorted_df.pivot_table(index=['OracleID','People ID', 'IEX_ID', 'Employee Name','Email Id','Alias','LOB','LOB_2','LOB_3', 'Supervisor Name','Wave','Detail Status','Status'],columns=[ 'Date'],values='First Shift',aggfunc='first').reset_index()
SCHEDULE = pd.DataFrame(SCHEDULE)

IEX_EXTEND = sorted_df[['Year', 'Month', 'Week Begin','Week_Monday', 'Date', 'Agent Name', 'IEX_ID','Datetime_Fluctuate_Start_Shift', 'Datetime_Fluctuate_End_Shift','Fluctuate Shift', 'Datetime_First_Start_Shift',
       'Datetime_First_End_Shift', 'First Shift', 'Time_Of_Day', 'Open Time', 'Extra Time', 'Target', 'Break Time','Lunch Time', 'Training', 'NCNS','AL','Unplanned','Planned', 'Night_Shift','Roster Presented', 'Roster Scheduled',
       'Shift Tracking', 'OT PreShift', 'OT PostShift', 'OracleID','People ID', 'Employee Name', 'Email Id', 'Alias', 'LOB','LOB_2','LOB_3','Supervisor Name', 'Wave', 'Detail Status', 'Status','OT Type','Combined OT Range']]

IEX_EXTEND = IEX_EXTEND.drop_duplicates()

In [29]:
for week_monday, group in IEX_EXTEND.groupby('Week_Monday'):
    week_monday_str = week_monday.strftime('%Y_%m_%d')
    file_name = f'{week_monday_str}.xlsx'
    file_path = os.path.join(folder_paths["output_iex"], file_name)
    group.to_excel(file_path, index=False)

for week_monday, group in INTERVALS_EXTEND.groupby('Week_Monday'):
    week_monday_str = week_monday.strftime('%Y-%m-%d')
    file_name = f'{week_monday_str}.xlsx'
    file_path = os.path.join(folder_paths["output_iex_intervals"], file_name)
    group.to_excel(file_path, index=False)

In [30]:
# import xlsxwriter
# from datetime import datetime

# all_sheets = {
#     '1. INTERVALS_EXTEND': INTERVALS_EXTEND,
#     '2. IEX_EXTEND': IEX_EXTEND,
#     '3. Schedule': SCHEDULE
# }

# today_temp = datetime.today().date()
# today = today_temp.strftime('%b_%d_%Y')

# writer = pd.ExcelWriter(f'{today}_IEX_DAILY_OUTPUT.xlsx', engine='xlsxwriter')

# for sheet_name in all_sheets.keys():
#     all_sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

# writer.close()